<a href="https://colab.research.google.com/github/bits05368/mlops_group53/blob/main/src/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from urllib.parse import urlparse
from utils import get_data_folder

# Paths
datafolder = get_data_folder()
TRAIN_FP = os.path.join(datafolder, "processed", "train.csv")
TEST_FP = os.path.join(datafolder, "processed", "test.csv")

def load_data():
    train_df = pd.read_csv(TRAIN_FP)
    test_df = pd.read_csv(TEST_FP)

    # X: all numeric features except target columns
    X_train = train_df.drop(columns=["target", "target_name"], errors="ignore")
    y_train = train_df["target"]
    X_test = test_df.drop(columns=["target", "target_name"], errors="ignore")
    y_test = test_df["target"]
    return X_train, X_test, y_train, y_test

def train_and_log_model(model_cls, params, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        model = model_cls(**params)
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        acc = accuracy_score(y_test, preds)

        # Log params & metrics
        mlflow.log_params(params)
        mlflow.log_metric("accuracy", acc)

        # Log model
        mlflow.sklearn.log_model(model, "model")

        print(f"{model_cls.__name__} accuracy: {acc:.4f}")
        return acc, mlflow.active_run().info.run_id

def main():
    mlflow.set_tracking_uri("http://127.0.0.1:5000")  # Change if needed
    mlflow.set_experiment("iris_experiment")

    X_train, X_test, y_train, y_test = load_data()

    results = []

    # Model 1: Logistic Regression
    lr_params = {"max_iter": 200}
    acc, run_id = train_and_log_model(LogisticRegression, lr_params,
                                      X_train, X_test, y_train, y_test)
    results.append(("LogisticRegression", acc, run_id))

    # Model 2: Random Forest
    rf_params = {"n_estimators": 100, "random_state": 42}
    acc, run_id = train_and_log_model(RandomForestClassifier, rf_params,
                                      X_train, X_test, y_train, y_test)
    results.append(("RandomForestClassifier", acc, run_id))

    # Pick best model
    best_model_name, best_acc, best_run_id = max(results, key=lambda x: x[1])
    print(f"Best model: {best_model_name} (acc={best_acc:.4f})")

    # Register best model in MLflow Model Registry
    # Works only if backend supports registry (e.g., file store, SQL, etc.)
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    if tracking_url_type_store != "file":  # registry not available with local file store
        model_uri = f"runs:/{best_run_id}/model"
        mlflow.register_model(model_uri, "IrisBestModel")
        print("Model registered as IrisBestModel")
    else:
        print("Model registry not supported with 'file' store backend.")

if __name__ == "__main__":
    main()